![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [4]:
from AlgorithmImports import *
from QuantConnect.Indicators import RelativeStrengthIndex, ExponentialMovingAverage, BollingerBands, MovingAverageConvergenceDivergence

# Create a QuantConnect Research environment
qb = QuantBook()

# Set the start date for historical data
qb.SetStartDate(2023, 10, 11)

# Add the SPY symbol
spy = qb.AddEquity("SPY")

# Request historical data with minute resolution
history = qb.History(qb.Securities.Keys, 360, Resolution.Minute)

# Create RSI indicator
rsi_period = 14
rsi = qb.Indicator(RelativeStrengthIndex(rsi_period), spy.Symbol, 360, Resolution.Minute)

# Create MACD indicator
macd_fast = 12
macd_slow = 26
macd_signal = 9
macd = qb.Indicator(MovingAverageConvergenceDivergence(macd_fast, macd_slow, macd_signal), spy.Symbol, 360, Resolution.Minute)

# Create Bollinger Bands indicator
bb_period = 20
bb = qb.Indicator(BollingerBands(bb_period, 2), spy.Symbol, 360, Resolution.Minute)

# Plot the indicators
rsi.plot()
macd.plot()
bb.plot()

In [15]:
symbol = qb.AddEquity("SPY").Symbol

bb = BollingerBands(20, 2)

bb_dataframe = qb.Indicator(bb, symbol, 50, Resolution.Daily)

history = qb.History[TradeBar](symbol, 70, Resolution.Daily)

window = {}
window['time'] = RollingWindow[DateTime](50)
window["bollingerbands"] = RollingWindow[float](50)
window["lowerband"] = RollingWindow[float](50)
window["middleband"] = RollingWindow[float](50)
window["upperband"] = RollingWindow[float](50)
window["bandwidth"] = RollingWindow[float](50)
window["percentb"] = RollingWindow[float](50)
window["standarddeviation"] = RollingWindow[float](50)
window["price"] = RollingWindow[float](50)

def UpdateBollingerBandWindow(sender: object, updated: IndicatorDataPoint) -> None:
    indicator = sender
    window['time'].Add(updated.EndTime)
    window["bollingerbands"].Add(updated.Value)
    window["lowerband"].Add(indicator.LowerBand.Current.Value)
    window["middleband"].Add(indicator.MiddleBand.Current.Value)
    window["upperband"].Add(indicator.UpperBand.Current.Value)
    window["bandwidth"].Add(indicator.BandWidth.Current.Value)
    window["percentb"].Add(indicator.PercentB.Current.Value)
    window["standarddeviation"].Add(indicator.StandardDeviation.Current.Value)
    window["price"].Add(indicator.Price.Current.Value)

bb.Updated += UpdateBollingerBandWindow

for bar in history:
    bb.Update(bar.EndTime, bar.Close)

#bb_dataframe = pd.DataFrame(window).set_index('time')

bb_dataframe

bb_plot = bb_dataframe[["upperband", "middleband", "lowerband", "price"]]
bb_plot.plot(figsize=(15, 10), title="SPY BB(20,2)")


In [13]:
symbol = qb.AddEquity("SPY").Symbol

bb = BollingerBands(20, 2)

bb_dataframe = qb.Indicator(bb, symbol, 50, Resolution.Daily)

bb_dataframe